In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

txn_train = pd.read_csv("transaction_train.csv")
txn_test = pd.read_csv("transaction_test.csv")
campaign = pd.read_csv("campaign_table.csv")
coupon = pd.read_csv("coupon_redempt.csv")
demo = pd.read_csv("demographic_new.csv")

print("txn_train:",txn_train.shape)
print("txn_test:",txn_test.shape)
print("campaign:",campaign.shape)
print("coupon:",coupon.shape)
print("demo:",demo.shape)

txn_train: (1783033, 13)
txn_test: (713927, 13)
campaign: (7208, 5)
coupon: (2318, 6)
demo: (2200, 8)


In [2]:
df_test= txn_test.copy()

In [3]:
df_test.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,DEPARTMENT
0,1060,26985040735,1,820162,1,1.79,315,0.00,1251,1,0.0,0.0,DRUG GM
1,1060,26985040735,1,827656,1,3.75,315,0.00,1251,1,0.0,0.0,DELI
2,1060,26985040735,1,829617,1,1.49,315,-0.50,1251,1,0.0,0.0,GROCERY
3,1060,26985040735,1,833014,1,2.79,315,-0.20,1251,1,0.0,0.0,SEAFOOD-PCKGD
4,1060,26985040735,1,833598,1,2.20,315,-0.44,1251,1,0.0,0.0,DELI


In [4]:
def get_indicator(d,k):
    if d == k:
        return 1
    else:
        return 0

In [5]:
for i in df_test.DEPARTMENT.unique():
    df_test[i+str("_ind")] = df_test.apply(lambda x: get_indicator(x.DEPARTMENT, i), axis =1)

In [6]:
df_test.shape

(713927, 50)

In [7]:
#df_test.to_csv("df_test.csv")

In [8]:
df_test.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,...,RESTAURANT_ind,POSTAL CENTER_ind,PROD-WHS SALES_ind,CNTRL/STORE SUP_ind,HBC_ind,PORK_ind,PHOTO_ind,VIDEO_ind,RX_ind,DELI/SNACK BAR_ind
0,1060,26985040735,1,820162,1,1.79,315,0.00,1251,1,...,0,0,0,0,0,0,0,0,0,0
1,1060,26985040735,1,827656,1,3.75,315,0.00,1251,1,...,0,0,0,0,0,0,0,0,0,0
2,1060,26985040735,1,829617,1,1.49,315,-0.50,1251,1,...,0,0,0,0,0,0,0,0,0,0
3,1060,26985040735,1,833014,1,2.79,315,-0.20,1251,1,...,0,0,0,0,0,0,0,0,0,0
4,1060,26985040735,1,833598,1,2.20,315,-0.44,1251,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df1= df_test.loc[:, ["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME","DEPARTMENT","QUANTITY","SALES_VALUE", "RETAIL_DISC"]].groupby(["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME","DEPARTMENT"], as_index=False)["QUANTITY","SALES_VALUE", "RETAIL_DISC"].sum()
df1

<ipython-input-9-7118d264471e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1= df_train.loc[:, ["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME","DEPARTMENT","QUANTITY","SALES_VALUE", "RETAIL_DISC"]].groupby(["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME","DEPARTMENT"], as_index=False)["QUANTITY","SALES_VALUE", "RETAIL_DISC"].sum()


,household_key,BASKET_ID,STORE_ID,DAY,WEEK_NO,TRANS_TIME,DEPARTMENT,QUANTITY,SALES_VALUE,RETAIL_DISC
0,2,28210649301,401,103,15,1904,GROCERY,7,14.29,-11.74
1,2,28210649301,401,103,15,1904,MEAT-PCKGD,1,3.29,0.00
2,2,28210649301,401,103,15,1904,PRODUCE,2,5.12,-5.03
3,2,28331488785,401,112,17,2117,GROCERY,2,4.38,-0.90
4,2,28391342107,296,117,17,2040,DRUG GM,1,2.86,0.00
...,...,...,...,...,...,...,...,...,...,...
193169,2499,41125020806,31782,628,90,1229,GROCERY,27,34.45,-0.67
193170,2499,41125191109,447,628,90,1338,DRUG GM,1,18.99,0.00
193171,2499,41125191109,447,628,90,1338,GROCERY,13,42.30,-5.56
193172,2499,41125191109,447,628,90,1338,MEAT-PCKGD,1,6.99,-3.00


In [11]:
df_1a=df1.loc[:, ["BASKET_ID","DEPARTMENT","QUANTITY"]].pivot(index='BASKET_ID', columns='DEPARTMENT', values='QUANTITY').reset_index()
df_1a.fillna(0, inplace = True)
df_1a.columns = ['BASKET_ID', ' _qty', 'AUTOMOTIVE_qty', 'CHEF_SHOPPE_qty', 'CNTRL_STORE_SUP_qty',
       'COSMETICS_qty', 'COUP/STR&MFG_qty', 'DAIRY_DELI_qty', 'DELI_qty', 'DELI_SNACK_BAR_qty',
       'DRUG_GM_qty', 'FLORAL_qty', 'FROZEN_GROCERY_qty', 'GARDEN_CENTER_qty', 'GM_MERCH_EXP_qty',
       'GROCERY_qty', 'HBC_qty', 'KIOSK_GAS_qty', 'MEAT_qty', 'MEAT_PCKGD_qty', 'MISC_SALES TRAN_qty',
       'MISC_TRANS_qty', 'NUTRITION_qty', 'PASTRY_qty', 'PHARMACY SUPPLY_qty', 'PHOTO_qty',
       'PORK_qty', 'POSTAL_CENTER_qty', 'PROD-WHS SALES_qty', 'PRODUCE_qty', 'RESTAURANT_qty',
       'RX_qty', 'SALAD_BAR_qty', 'SEAFOOD_qty', 'SEAFOOD_PCKGD_qty', 'SPIRITS_qty',
       'TRAVEL&LEISUR_qty', 'VIDEO_qty']
df_1a

,BASKET_ID,_qty,AUTOMOTIVE_qty,CHEF_SHOPPE_qty,CNTRL_STORE_SUP_qty,COSMETICS_qty,COUP/STR&MFG_qty,DAIRY_DELI_qty,DELI_qty,DELI_SNACK_BAR_qty,...,PROD-WHS SALES_qty,PRODUCE_qty,RESTAURANT_qty,RX_qty,SALAD_BAR_qty,SEAFOOD_qty,SEAFOOD_PCKGD_qty,SPIRITS_qty,TRAVEL&LEISUR_qty,VIDEO_qty
0,26985040735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,26985360571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,26996776017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26996870743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26997328096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77684,41337964158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77685,41338366916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77686,41338368819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77687,41338369685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_1b=df1.loc[:, ["BASKET_ID","DEPARTMENT","SALES_VALUE"]].pivot(index='BASKET_ID', columns='DEPARTMENT', values='SALES_VALUE').reset_index()
df_1b.fillna(0, inplace = True)
df_1b.columns = ['BASKET_ID', ' _sales', 'AUTOMOTIVE_sales', 'CHEF_SHOPPE_sales', 'CNTRL_STORE_SUP_sales',
       'COSMETICS_sales', 'COUP/STR&MFG_sales', 'DAIRY_DELI_sales', 'DELI_sales', 'DELI_SNACK_BAR_sales',
       'DRUG_GM_sales', 'FLORAL_sales', 'FROZEN_GROCERY_sales', 'GARDEN_CENTER_sales', 'GM_MERCH_EXP_sales',
       'GROCERY_sales', 'HBC_sales', 'KIOSK_GAS_sales', 'MEAT_sales', 'MEAT_PCKGD_sales', 'MISC_SALES TRAN_sales',
       'MISC_TRANS_sales', 'NUTRITION_sales', 'PASTRY_sales', 'PHARMACY SUPPLY_sales', 'PHOTO_sales',
       'PORK_sales', 'POSTAL_CENTER_sales', 'PROD-WHS SALES_sales', 'PRODUCE_sales', 'RESTAURANT_sales',
       'RX_sales', 'SALAD_BAR_sales', 'SEAFOOD_sales', 'SEAFOOD_PCKGD_sales', 'SPIRITS_sales',
       'TRAVEL&LEISUR_sales', 'VIDEO_sales']
df_1b

,BASKET_ID,_sales,AUTOMOTIVE_sales,CHEF_SHOPPE_sales,CNTRL_STORE_SUP_sales,COSMETICS_sales,COUP/STR&MFG_sales,DAIRY_DELI_sales,DELI_sales,DELI_SNACK_BAR_sales,...,PROD-WHS SALES_sales,PRODUCE_sales,RESTAURANT_sales,RX_sales,SALAD_BAR_sales,SEAFOOD_sales,SEAFOOD_PCKGD_sales,SPIRITS_sales,TRAVEL&LEISUR_sales,VIDEO_sales
0,26985040735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.52,0.0,...,0.0,3.92,0.0,0.0,0.0,0.0,2.79,0.0,0.0,0.0
1,26985360571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,2.08,0.0,0.0,0.0,0.0,3.21,0.0,0.0,0.0
2,26996776017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,26996870743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.59,0.0,...,0.0,8.93,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,26997328096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.50,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77684,41337964158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77685,41338366916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77686,41338368819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,3.06,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77687,41338369685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [13]:
df_1c=df1.loc[:, ["BASKET_ID","DEPARTMENT","RETAIL_DISC"]].pivot(index='BASKET_ID', columns='DEPARTMENT', values='RETAIL_DISC').reset_index()
df_1c.fillna(0, inplace = True)
df_1c.columns =['BASKET_ID', ' _retail_disc', 'AUTOMOTIVE_retail_disc', 'CHEF_SHOPPE_retail_disc', 'CNTRL_STORE_SUP_retail_disc',
       'COSMETICS_retail_disc', 'COUP/STR&MFG_retail_disc', 'DAIRY_DELI_retail_disc', 'DELI_retail_disc', 'DELI_SNACK_BAR_retail_disc',
       'DRUG_GM_retail_disc', 'FLORAL_retail_disc', 'FROZEN_GROCERY_retail_disc', 'GARDEN_CENTER_retail_disc', 'GM_MERCH_EXP_retail_disc',
       'GROCERY_retail_disc', 'HBC_retail_disc', 'KIOSK_GAS_retail_disc', 'MEAT_retail_disc', 'MEAT_PCKGD_retail_disc', 'MISC_SALES TRAN_retail_disc',
       'MISC_TRANS_retail_disc', 'NUTRITION_retail_disc', 'PASTRY_retail_disc', 'PHARMACY SUPPLY_retail_disc', 'PHOTO_retail_disc',
       'PORK_retail_disc', 'POSTAL_CENTER_retail_disc', 'PROD-WHS SALES_retail_disc', 'PRODUCE_retail_disc', 'RESTAURANT_retail_disc',
       'RX_retail_disc', 'SALAD_BAR_retail_disc', 'SEAFOOD_retail_disc', 'SEAFOOD_PCKGD_retail_disc', 'SPIRITS_retail_disc',
       'TRAVEL&LEISUR_retail_disc', 'VIDEO_retail_disc']
df_1c

,BASKET_ID,_retail_disc,AUTOMOTIVE_retail_disc,CHEF_SHOPPE_retail_disc,CNTRL_STORE_SUP_retail_disc,COSMETICS_retail_disc,COUP/STR&MFG_retail_disc,DAIRY_DELI_retail_disc,DELI_retail_disc,DELI_SNACK_BAR_retail_disc,...,PROD-WHS SALES_retail_disc,PRODUCE_retail_disc,RESTAURANT_retail_disc,RX_retail_disc,SALAD_BAR_retail_disc,SEAFOOD_retail_disc,SEAFOOD_PCKGD_retail_disc,SPIRITS_retail_disc,TRAVEL&LEISUR_retail_disc,VIDEO_retail_disc
0,26985040735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.44,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,-0.20,0.0,0.0,0.0
1,26985360571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,-0.30,0.0,0.0,0.0,0.0,-1.58,0.0,0.0,0.0
2,26996776017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,26996870743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.40,0.0,...,0.0,-0.99,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,26997328096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77684,41337964158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77685,41338366916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77686,41338368819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77687,41338369685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [14]:
df_ = pd.merge(df_1a, df_1b, on = "BASKET_ID", how = "inner")
df_dep_agg_hh = pd.merge(df_, df_1c, on = "BASKET_ID", how = "inner")
df_dep_agg_hh

,BASKET_ID,_qty,AUTOMOTIVE_qty,CHEF_SHOPPE_qty,CNTRL_STORE_SUP_qty,COSMETICS_qty,COUP/STR&MFG_qty,DAIRY_DELI_qty,DELI_qty,DELI_SNACK_BAR_qty,...,PROD-WHS SALES_retail_disc,PRODUCE_retail_disc,RESTAURANT_retail_disc,RX_retail_disc,SALAD_BAR_retail_disc,SEAFOOD_retail_disc,SEAFOOD_PCKGD_retail_disc,SPIRITS_retail_disc,TRAVEL&LEISUR_retail_disc,VIDEO_retail_disc
0,26985040735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,-0.20,0.0,0.0,0.0
1,26985360571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.30,0.0,0.0,0.0,0.0,-1.58,0.0,0.0,0.0
2,26996776017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,26996870743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,-0.99,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,26997328096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77684,41337964158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77685,41338366916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77686,41338368819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
77687,41338369685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [15]:
df2= df_test.drop(["PRODUCT_ID", "DEPARTMENT"], axis = 1).groupby(["household_key","BASKET_ID","STORE_ID",  "DAY","WEEK_NO", "TRANS_TIME"], as_index=False).sum()
df2

,household_key,BASKET_ID,STORE_ID,DAY,WEEK_NO,TRANS_TIME,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,...,RESTAURANT_ind,POSTAL CENTER_ind,PROD-WHS SALES_ind,CNTRL/STORE SUP_ind,HBC_ind,PORK_ind,PHOTO_ind,VIDEO_ind,RX_ind,DELI/SNACK BAR_ind
0,2,28210649301,401,103,15,1904,10,22.70,-16.77,-2.5,...,0,0,0,0,0,0,0,0,0,0
1,2,28331488785,401,112,17,2117,2,4.38,-0.90,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,28391342107,296,117,17,2040,5,12.84,-1.78,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2,28405847708,401,118,18,1749,38,30.85,-7.30,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2,28517296871,3278,121,18,2043,10,26.94,-4.68,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77684,2499,40911571508,447,617,89,1746,3,13.17,-1.00,0.0,...,0,0,0,0,0,0,0,0,0,0
77685,2499,40968776218,447,620,89,1438,28,74.89,-8.95,0.0,...,0,0,0,0,0,0,0,0,0,0
77686,2499,41026287236,447,622,90,2132,3,8.07,-0.20,0.0,...,0,0,0,0,0,0,0,0,0,0
77687,2499,41125020806,31782,628,90,1229,27,34.45,-0.67,0.0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
aggregated_data = pd.merge(df2, df_dep_agg_hh, on = "BASKET_ID", how = "inner")
aggregated_data.shape

(77689, 159)

In [17]:
aggregated_data.head()

,household_key,BASKET_ID,STORE_ID,DAY,WEEK_NO,TRANS_TIME,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,...,PROD-WHS SALES_retail_disc,PRODUCE_retail_disc,RESTAURANT_retail_disc,RX_retail_disc,SALAD_BAR_retail_disc,SEAFOOD_retail_disc,SEAFOOD_PCKGD_retail_disc,SPIRITS_retail_disc,TRAVEL&LEISUR_retail_disc,VIDEO_retail_disc
0,2,28210649301,401,103,15,1904,10,22.70,-16.77,-2.5,...,0.0,-5.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,28331488785,401,112,17,2117,2,4.38,-0.90,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,28391342107,296,117,17,2040,5,12.84,-1.78,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,28405847708,401,118,18,1749,38,30.85,-7.30,0.0,...,0.0,-0.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,28517296871,3278,121,18,2043,10,26.94,-4.68,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df3 = df_test.groupby(["household_key","BASKET_ID","STORE_ID"], as_index = False)["PRODUCT_ID"].nunique()
df3.columns = ["household_key","BASKET_ID","STORE_ID", "count_diff_items_bought"] #sum of qty : total items bought
df3

,household_key,BASKET_ID,STORE_ID,count_diff_items_bought
0,2,28210649301,401,9
1,2,28331488785,401,2
2,2,28391342107,296,4
3,2,28405847708,401,19
4,2,28517296871,3278,10
...,...,...,...,...
77684,2499,40911571508,447,3
77685,2499,40968776218,447,25
77686,2499,41026287236,447,3
77687,2499,41125020806,31782,16


In [20]:
aggregated_data = aggregated_data.merge(df3)
aggregated_data.shape

(77689, 160)

In [21]:
df4 = df_test.groupby(["household_key","BASKET_ID","STORE_ID"], as_index = False)["DEPARTMENT"].nunique()
df4.columns = ["household_key","BASKET_ID","STORE_ID", "no_of_depts_shopped"]
df4

,household_key,BASKET_ID,STORE_ID,no_of_depts_shopped
0,2,28210649301,401,3
1,2,28331488785,401,1
2,2,28391342107,296,2
3,2,28405847708,401,5
4,2,28517296871,3278,3
...,...,...,...,...
77684,2499,40911571508,447,2
77685,2499,40968776218,447,4
77686,2499,41026287236,447,2
77687,2499,41125020806,31782,1


In [22]:
aggregated_data = aggregated_data.merge(df4)
aggregated_data.shape

(77689, 161)

In [23]:
#import pickle
#pickle.dump(aggregated_data, open('Desktop', 'wb'))

# aggregating at hh level

In [26]:
aggregated_data_21m = aggregated_data[aggregated_data.DAY < 621]
aggregated_data_21m.shape

(75181, 161)

In [27]:
temp = aggregated_data_21m.loc[:, ["household_key","BASKET_ID", "NUTRITION_ind","DAY"]]
temp1 = temp[temp.NUTRITION_ind > 0]
temp1 = temp1.groupby(["household_key"], as_index = False)["DAY"].last()
temp1.columns = ["household_key", "last_day_nutr"]
temp1["recency_nutr"] = 620 - temp1["last_day_nutr"]
df5 = temp1.loc[:, ["household_key", "recency_nutr"]]
df5

,household_key,recency_nutr
0,2,142
1,6,38
2,8,321
3,16,209
4,18,94
...,...,...
501,2471,234
502,2477,492
503,2487,208
504,2488,55


In [28]:
temp = aggregated_data_21m.loc[:, ["household_key","BASKET_ID","NUTRITION_ind","DAY"]]
temp1 = temp[temp.NUTRITION_ind > 0]
temp1["lagged"] = temp1.groupby(["household_key"])['DAY'].shift(1)
temp1["diff"] = temp1["DAY"] - temp1["lagged"]
tmp1 = temp1.dropna(subset = ["diff"])
df6 = tmp1.groupby(["household_key"], as_index = False)["diff"].first()
df6.columns = ["household_key", "days_to_second_order_nut"]
df6

<ipython-input-28-41026cddb578>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1["lagged"] = temp1.groupby(["household_key"])['DAY'].shift(1)
<ipython-input-28-41026cddb578>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1["diff"] = temp1["DAY"] - temp1["lagged"]


,household_key,days_to_second_order_nut
0,2,10.0
1,6,29.0
2,16,14.0
3,18,3.0
4,35,349.0
...,...,...
377,2469,57.0
378,2470,43.0
379,2487,1.0
380,2488,21.0


In [29]:
temp2 = temp1.loc[:, ["household_key", "diff"]]
df7 = temp2.groupby(["household_key"], as_index = False).mean()
df7.columns = ["household_key", "avg_gap_in_orders_nut"]
df7

,household_key,avg_gap_in_orders_nut
0,2,157.000000
1,6,39.888889
2,8,NaN
3,16,47.000000
4,18,26.062500
...,...,...
501,2471,NaN
502,2477,NaN
503,2487,18.166667
504,2488,36.230769


In [30]:
min_max_days= aggregated_data_21m.loc[:, ["household_key","DAY"]].groupby(["household_key"], as_index = False).agg(["min", "max"])
min_max_days = min_max_days.reset_index()
min_max_days.columns = ["household_key", "min_day", "max_day"]
min_max_days["tenure"] = min_max_days["max_day"]-min_max_days["min_day"]
min_max_df = min_max_days.loc[:, ["household_key", "tenure"]]

In [31]:
drop_cols = ["BASKET_ID", "STORE_ID", "DAY", "WEEK_NO", "TRANS_TIME"]
aggregated_data_21m_1 = aggregated_data_21m.drop(drop_cols,axis =1)
aggregated_data_21m_2 = aggregated_data_21m_1.groupby(["household_key"], as_index = False).sum()
aggregated_data_21m_2.shape

(750, 156)

In [32]:
df_merge1 = pd.merge(aggregated_data_21m_2, df5, on = "household_key", how = "left")
df_merge2 = pd.merge(df_merge1, df6, on = "household_key", how = "left")
df_merge3 = pd.merge(df_merge2, df7, on = "household_key", how = "left")
df_merge4 = pd.merge(df_merge3, min_max_df, on = "household_key", how = "left")
df_merge4.shape

(750, 160)

In [33]:
df_merge4.head()

,household_key,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,DRUG GM_ind,DELI_ind,GROCERY_ind,SEAFOOD-PCKGD_ind,...,SEAFOOD_PCKGD_retail_disc,SPIRITS_retail_disc,TRAVEL&LEISUR_retail_disc,VIDEO_retail_disc,count_diff_items_bought,no_of_depts_shopped,recency_nutr,days_to_second_order_nut,avg_gap_in_orders_nut,tenure
0,2,722,1740.67,-306.09,-9.00,0.00,91,12,398,1,...,0.00,0.0,0.0,0.0,620,138,142.0,10.0,157.000000,517
1,6,94399,5301.47,-726.48,-3.55,-0.45,198,19,594,4,...,-13.00,0.0,0.0,0.0,1585,605,38.0,29.0,39.888889,502
2,8,67794,4387.35,-853.24,-27.05,-1.55,164,15,1057,5,...,-6.30,0.0,0.0,0.0,1540,334,321.0,NaN,NaN,545
3,16,81186,1486.02,-170.20,0.00,0.00,33,28,333,0,...,0.00,0.0,0.0,0.0,506,213,209.0,14.0,47.000000,515
4,18,182710,5561.20,-824.63,-47.58,-14.10,54,72,992,21,...,-91.22,0.0,0.0,0.0,1554,354,94.0,3.0,26.062500,524


### adding demo data

In [34]:
demo

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,15-24K,Homeowner,Unknown,1,None/Unknown,3
2,19-24,B,50-74K,Homeowner,1 Adult Kids,2,None/Unknown,4
3,25-34,A,35-49K,Unknown,2 Adults Kids,3,None/Unknown,5
4,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
...,...,...,...,...,...,...,...,...
2195,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
2196,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
2197,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498
2198,25-34,U,Under 15K,Unknown,2 Adults Kids,3,1,2499


In [35]:
one_hot_encoded_data = pd.get_dummies(demo, columns =['AGE_DESC', 'MARITAL_STATUS_CODE', 'INCOME_DESC', 'HOMEOWNER_DESC',
       'HH_COMP_DESC', 'HOUSEHOLD_SIZE_DESC', 'KID_CATEGORY_DESC'])
one_hot_encoded_data

,household_key,AGE_DESC_19-24,AGE_DESC_25-34,AGE_DESC_35-44,AGE_DESC_45-54,AGE_DESC_55-64,AGE_DESC_65+,MARITAL_STATUS_CODE_A,MARITAL_STATUS_CODE_B,MARITAL_STATUS_CODE_U,...,HH_COMP_DESC_Unknown,HOUSEHOLD_SIZE_DESC_1,HOUSEHOLD_SIZE_DESC_2,HOUSEHOLD_SIZE_DESC_3,HOUSEHOLD_SIZE_DESC_4,HOUSEHOLD_SIZE_DESC_5+,KID_CATEGORY_DESC_1,KID_CATEGORY_DESC_2,KID_CATEGORY_DESC_3+,KID_CATEGORY_DESC_None/Unknown
0,1,0,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,1
1,3,0,0,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,1
2,4,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
3,5,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
4,7,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,2496,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0
2196,2497,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
2197,2498,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2198,2499,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0


In [53]:
df_merge4_demo = pd.merge(df_merge4, one_hot_encoded_data, on = "household_key", how = "left")
df_merge4_demo.shape

(750, 201)

## adding coupon

In [55]:
coupon_nut = coupon[coupon.Department == "NUTRITION"]
coupon_nut["coupon_nut_red"] = 1
coupon_nut_df = coupon_nut.loc[:, ["household_key","coupon_nut_red" ]]

<ipython-input-55-a2bb3a5f8a2e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupon_nut["coupon_nut_red"] = 1


In [59]:
df_merge4_demo_wc = pd.merge(df_merge4_demo, coupon_nut_df, on = "household_key", how = "left")
df_merge4_demo_wc = df_merge4_demo_wc.drop_duplicates()

In [60]:
df_merge4_demo_wc.shape

(750, 202)

In [39]:
#creating a target variable

In [40]:
aggregated_data_3m = aggregated_data[aggregated_data.DAY >= 621]
aggregated_data_3m.shape

(2508, 161)

In [41]:
get_target = aggregated_data_3m.loc[:, ["household_key","BASKET_ID", "NUTRITION_ind"]]
get_target.head()

,household_key,BASKET_ID,NUTRITION_ind
43,2,41008217091,0
260,6,41063027484,0
261,6,41124804365,0
262,6,41160383039,0
263,6,41238687971,0


In [42]:
get_target1= get_target.groupby(["household_key"], as_index = False)["NUTRITION_ind"].sum()

In [43]:
get_target1["Target"] = get_target1["NUTRITION_ind"].apply(lambda x: 1 if x > 0 else 0)
get_target1

,household_key,NUTRITION_ind,Target
0,2,0,0
1,6,0,0
2,8,0,0
3,16,0,0
4,18,0,0
...,...,...,...
575,2474,0,0
576,2477,5,1
577,2478,0,0
578,2488,0,0


In [44]:
y_test = get_target1.loc[:,["household_key","Target"]]
y_test.shape

(580, 2)

In [45]:
y_test.Target.value_counts()

0    483
1     97
Name: Target, dtype: int64

### final merge with Target

In [61]:
X_test = df_merge4_demo_wc.copy()
X_test.shape

(750, 202)

In [62]:
test_finaldf = pd.merge(df_merge4_demo_wc, y_train, on = "household_key", how = "left")
test_finaldf.shape

(750, 203)

In [63]:
test_finaldf.Target.value_counts()

0.0    483
1.0     97
Name: Target, dtype: int64

In [64]:
y_test.head(2)

,household_key,Target
0,2,0
1,6,0


In [65]:
test_finaldf.head()

,household_key,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,DRUG GM_ind,DELI_ind,GROCERY_ind,SEAFOOD-PCKGD_ind,...,HOUSEHOLD_SIZE_DESC_2,HOUSEHOLD_SIZE_DESC_3,HOUSEHOLD_SIZE_DESC_4,HOUSEHOLD_SIZE_DESC_5+,KID_CATEGORY_DESC_1,KID_CATEGORY_DESC_2,KID_CATEGORY_DESC_3+,KID_CATEGORY_DESC_None/Unknown,coupon_nut_red,Target
0,2,722,1740.67,-306.09,-9.00,0.00,91,12,398,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,6,94399,5301.47,-726.48,-3.55,-0.45,198,19,594,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,8,67794,4387.35,-853.24,-27.05,-1.55,164,15,1057,5,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0
3,16,81186,1486.02,-170.20,0.00,0.00,33,28,333,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0
4,18,182710,5561.20,-824.63,-47.58,-14.10,54,72,992,21,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0


In [66]:
#test_finaldf.to_csv("test_df_21m.csv")
#test_finaldf.to_csv("test_df_21m_v2.csv")